In [1]:
import os
import numpy as np
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

In [2]:
print(tf.__version__)

2.5.0


In [3]:
dataset_name='ml-100k'
dataset_folder='ml'
preflix_folder='11_17_new_settings'

In [4]:
import platform
if platform.system().lower() == 'windows':
    import pickle5 as pickle
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_all_ratings_mapped_index.pkl', 'rb') as f:  
        user_dict = pickle.load(f)
    print("windows")
elif platform.system().lower() == 'linux':
    import pickle
    with open('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_all_ratings_mapped_index.pkl', 'rb') as f:  
        user_dict = pickle.load(f)
    print("linux")
items_num=[]
for i in user_dict.values():
    items_num.append(max(i))
items_num=max(items_num)

len_seq=10


batch_size=512
epoch_num=100
hidden_size=64
keep_rate=0.9
layers_num=3

interest_num=3
kernel_size=4
transformerencoder=0

neg_num=5
test_neg_num=1000

windows


In [5]:
if platform.system().lower() == 'windows':
    print("windows")
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_'+str(len_seq)+'_seq_train_data.pkl', 'rb') as f:
        train_data=pickle.load(f)
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_'+str(len_seq)+'_seq_test_data.pkl', 'rb') as f:
        test_data=pickle.load(f)
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_'+str(len_seq)+'_seq_val_data.pkl', 'rb') as f:
        val_data=pickle.load(f)
elif platform.system().lower() == 'linux':
    print("linux")
    with open('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_'+str(len_seq)+'_seq_train_data.pkl', 'rb') as f:
        train_data=pickle.load(f)
    with open('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_'+str(len_seq)+'_seq_test_data.pkl', 'rb') as f:
        test_data=pickle.load(f)
    with open('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_'+str(len_seq)+'_seq_val_data.pkl', 'rb') as f:
        val_data=pickle.load(f)

windows


In [6]:
print(np.shape(train_data['click_seq']),np.shape(val_data['click_seq']),np.shape(test_data['click_seq']))

(51854, 10) (6969, 10) (14264, 10)


In [7]:
def scaled_dot_product_attention(q, k, v, mask):
    """Attention Mechanism Function.
    Args:
        :param q: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param k: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param v: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param mask: A 3d/4d tensor with shape of (None, ..., seq_len, 1)
    :return:
    """
    mat_qk = tf.matmul(q, k, transpose_b=True)  # (None, seq_len, seq_len)
    # Scaled
    dk = tf.cast(k.shape[-1], dtype=tf.float32)
    scaled_att_logits = mat_qk / tf.sqrt(dk)

    paddings = tf.ones_like(scaled_att_logits) * (-2 ** 32 + 1)  # (None, seq_len, seq_len)
    if mask!=None:
        outputs = tf.where(tf.equal(mask, tf.zeros_like(mask)), paddings, scaled_att_logits)  # (None, seq_len, seq_len)
    else:
        outputs=scaled_att_logits
    # softmax
    outputs = tf.nn.softmax(logits=outputs)  # (None, seq_len, seq_len)
    outputs = tf.matmul(outputs, v)  # (None, seq_len, dim)

    return outputs


def split_heads(x, seq_len, num_heads, depth):
    """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    Args:
        :param x: A Tensor with shape of [batch_size, seq_len, num_heads * depth]
        :param seq_len: A scalar(int).
        :param num_heads: A scalar(int).
        :param depth: A scalar(int).
    :return: A tensor with shape of [batch_size, num_heads, seq_len, depth]
    """
    x = tf.reshape(x, (-1, seq_len, num_heads, depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

def normalize_adj_tensor(adj, seq_len):
    adj = adj + tf.expand_dims(tf.eye(seq_len), axis=0)
    rowsum = tf.reduce_sum(adj, axis=1)
    d_inv_sqrt = tf.pow(rowsum, -0.5)
    candidate_a = tf.zeros_like(d_inv_sqrt)
    d_inv_sqrt = tf.where(tf.math.is_inf(d_inv_sqrt), candidate_a, d_inv_sqrt)
    d_mat_inv_sqrt = tf.linalg.diag(d_inv_sqrt)
    norm_adg = tf.matmul(d_mat_inv_sqrt, adj)
    return norm_adg

In [8]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        """Multi Head Attention Mechanism.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads. If num_heads == 1, the layer is a single self-attention layer.
        :return:
        """
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        self.wq = tf.keras.layers.Dense(d_model, activation=None)
        self.wk = tf.keras.layers.Dense(d_model, activation=None)
        self.wv = tf.keras.layers.Dense(d_model, activation=None)

    def call(self, q, k, v, mask):
        q = self.wq(q)  # (None, seq_len, d_model)
        k = self.wk(k)  # (None, seq_len, d_model)
        v = self.wv(v)  # (None, seq_len, d_model)
        # split d_model into num_heads * depth
        seq_len, d_model = q.shape[1], q.shape[2]
        q = split_heads(q, seq_len, self.num_heads, q.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        k = split_heads(k, seq_len, self.num_heads, k.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        v = split_heads(v, seq_len, self.num_heads, v.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        # mask
        if mask!=None:
            mask = tf.tile(tf.expand_dims(mask, axis=1), [1, self.num_heads, 1, 1])  # (None, num_heads, seq_len, 1)
        # attention
        scaled_attention = scaled_dot_product_attention(q, k, v, mask)  # (None, num_heads, seq_len, d_model // num_heads)
        # reshape
        outputs = tf.reshape(tf.transpose(scaled_attention, [0, 2, 1, 3]), [-1, seq_len, d_model])  # (None, seq_len, d_model)
        return outputs


class FFN(tf.keras.layers.Layer):
    def __init__(self, hidden_unit, d_model):
        """Feed Forward Network.
        Args:
            :param hidden_unit: A scalar.
            :param d_model: A scalar.
        :return:
        """
        super(FFN, self).__init__()
        self.conv1 = tf.keras.layers.Conv1D(filters=hidden_unit, kernel_size=1, activation='relu', use_bias=True)
        self.conv2 = tf.keras.layers.Conv1D(filters=d_model, kernel_size=1, activation=None, use_bias=True)

    def call(self, inputs):
        x = self.conv1(inputs)
        output = self.conv2(x)
        return output


class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads=1, ffn_hidden_unit=128, dropout=0., layer_norm_eps=1e-6):
        """Encoder Layer.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads.
            :param ffn_hidden_unit: A scalar. Number of hidden unit in FFN
            :param dropout: A scalar. Number of dropout.
            :param layer_norm_eps: A scalar. Small float added to variance to avoid dividing by zero.
        :return:
        """
        super(TransformerEncoder, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = FFN(ffn_hidden_unit, d_model)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, inputs):
        x, mask= inputs
        # self-attention
        att_out = self.mha(x, x, x, mask)  # (None, seq_len, d_model)
        att_out = self.dropout1(att_out)
        # residual add
        out1 = self.layernorm1(x + att_out)  # (None, seq_len, d_model)
        # ffn
        ffn_out = self.ffn(out1)
        ffn_out = self.dropout2(ffn_out)
        # residual add
        out2 = self.layernorm2(out1 + ffn_out)  # (None, seq_len, d_model)
        return out2

class gcn_layer(tf.keras.layers.Layer):
    def __init__(self, hidden_size, num_layer):
        super(gcn_layer, self).__init__()
        #weights_size_list = [hidden_size,64,64,64]
        #all_weights = {}
        self.num_layer=num_layer
        self.conv_dense=tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
    
    def call(self,A, x):
    # gcn has three layers
        all_embeddings =[x]
        for k in range(self.num_layer):
            embeddings = tf.matmul(A, x)
            embeddings = self.conv_dense(embeddings)
            all_embeddings.append(embeddings)
        return all_embeddings

class light_gcn_layer(tf.keras.layers.Layer):
    def __init__(self, hidden_size, num_layer):
        super(gcn_layer, self).__init__()
        #weights_size_list = [hidden_size,64,64,64]
        #all_weights = {}
        self.num_layer=num_layer
        self.conv_dense=tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
    
    def call(self,A, x):
    # gcn has three layers
        all_embeddings =[x]
        for k in range(self.num_layer):
            embeddings = tf.matmul(A, x)
            #embeddings = self.conv_dense(embeddings)
            all_embeddings.append(embeddings)
        return all_embeddings    
    
class hierarchical_interest(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        ###-------------------------------###
    def call(self, ori_all_embeddings):   
        ###-------------------------------###
        return all_embeddings

class CapsuleNetwork(tf.keras.layers.Layer):
    def __init__(self, embed_dim, seq_len, bilinear_type=0, num_interest=4, stop_grad=True):
        super(CapsuleNetwork, self).__init__()
        self.bilinear_type = bilinear_type
        self.seq_len = seq_len
        self.num_interest = num_interest
        self.embed_dim = embed_dim
        self.bi_dense=tf.keras.layers.Dense(hidden_size * self.num_interest, activation=None,use_bias=False)
        self.lstm=tf.keras.layers.LSTM(self.embed_dim*2, unit_forget_bias=1.0,return_sequences=True)
        self.stop_grad = stop_grad

    def build(self, input_shape):
        if self.bilinear_type >= 2:
            self.w = self.add_weight(
                shape=[1, self.seq_len, self.num_interest * self.embed_dim, self.embed_dim],
                initializer='random_normal',
                name='weights'
            )

    def call(self, hist_emb, mask):
        if self.bilinear_type == 0:
            hist_emb_hat = tf.tile(hist_emb, [1, 1, self.num_interest])  # (None, seq_len, num_inter * embed_dim)
        elif self.bilinear_type == 1:
            #outputs = self.lstm(hist_emb)
            #output = tf.concat(outputs, axis=0)
            #output = tf.reshape(output, (-1, self.seq_len, self.embed_dim*2))
            #output = tf.layers.dense(output, self.embed_dim*self.num_interest, activation=None, use_bias=False)
            hist_emb_hat = self.bi_dense(hist_emb)
        else:
            u = tf.expand_dims(hist_emb, axis=2)  # (None, seq_len, 1, embed_dim)
            hist_emb_hat = tf.reduce_sum(self.w * u, axis=3)  # (None, seq_len, num_inter * embed_dim)
        hist_emb_hat = tf.reshape(hist_emb_hat, [-1, self.seq_len, self.num_interest, self.embed_dim])
        hist_emb_hat = tf.transpose(hist_emb_hat, [0, 2, 1, 3])  # (None, num_inter, seq_len, embed_dim)
        hist_emb_hat = tf.reshape(hist_emb_hat, [-1, self.num_interest, self.seq_len, self.embed_dim])
        if self.stop_grad:
            hist_emb_iter = tf.stop_gradient(hist_emb_hat)
        else:
            hist_emb_iter = hist_emb_hat  # (None, num_inter, seq_len, embed_dim)

        if self.bilinear_type > 0:
            '''self.capsule_weight = self.add_weight(
                shape=[tf.shape(hist_emb_hat)[0], self.num_interest, self.seq_len],
                initializer=tf.zeros_initializer())'''
            self.capsule_weight = tf.random.truncated_normal(
                shape=[tf.shape(hist_emb_hat)[0], self.num_interest, self.seq_len],
                stddev=1.0)
            
        else:
            self.capsule_weight = tf.random.truncated_normal(
                shape=[tf.shape(hist_emb_hat)[0], self.num_interest, self.seq_len],
                stddev=1.0)
        tf.stop_gradient(self.capsule_weight)

        for i in range(3):
            att_mask = tf.tile(tf.expand_dims(mask, axis=1), [1, self.num_interest, 1])  # (None, num_inter, seq_len)
            paddings = tf.zeros_like(att_mask)

            capsule_softmax_weight = tf.nn.softmax(self.capsule_weight, axis=1)  # (None, num_inter, seq_len)
            capsule_softmax_weight = tf.where(tf.equal(att_mask, 0), paddings, capsule_softmax_weight)
            capsule_softmax_weight = tf.expand_dims(capsule_softmax_weight, 2)  # (None, num_inter, 1, seq_len)

            if i < 2:
                interest_capsule = tf.matmul(capsule_softmax_weight, hist_emb_iter)  # (None, num_inter, 1, embed_dim)
                cap_norm = tf.reduce_sum(tf.square(interest_capsule), -1, keepdims=True)
                scalar_factor = cap_norm / (1 + cap_norm) / tf.sqrt(cap_norm + 1e-9)
                interest_capsule = scalar_factor * interest_capsule

                delta_weight = tf.matmul(hist_emb_iter, tf.transpose(interest_capsule, [0, 1, 3, 2]))
                delta_weight = tf.reshape(delta_weight, [-1, self.num_interest, self.seq_len])
                self.capsule_weight = self.capsule_weight + delta_weight
            else:
                interest_capsule = tf.matmul(capsule_softmax_weight, hist_emb_hat)
                cap_norm = tf.reduce_sum(tf.square(interest_capsule), -1, True)
                scalar_factor = cap_norm / (1 + cap_norm) / tf.sqrt(cap_norm + 1e-9)
                interest_capsule = scalar_factor * interest_capsule

        interest_capsule = tf.reshape(interest_capsule, [-1, self.num_interest, self.embed_dim])
        return interest_capsule

In [9]:
class fcn_net(tf.keras.layers.Layer):
    def __init__(self,layer_sizes=[100,64]):
        super(fcn_net, self).__init__()
        self.hidden_layer={}
        self.hidden_layer['layer_0']=tf.keras.layers.Dense(layer_sizes[0],activation='relu')
        self.hidden_layer['layer_1']=tf.keras.layers.Dense(layer_sizes[1],activation='relu')
        self.bn=[tf.keras.layers.BatchNormalization(
                            momentum=0.95,
                            epsilon=0.0001
                    ),
                 tf.keras.layers.BatchNormalization(
                            momentum=0.95,
                            epsilon=0.0001
                    )]
        self.last_out=tf.keras.layers.Dense(1)
    def call(self, model_output, layer_sizes=[100,64], enable_BN=True):
        last_layer_size = tf.shape(model_output)[-1]
        layer_idx = 0
        hidden_nn_layers = []
        hidden_nn_layers.append(model_output)
        for idx, layer_size in enumerate(layer_sizes):
            curr_hidden_nn_layer=self.hidden_layer['layer_'+str(idx)](hidden_nn_layers[idx])
            if enable_BN is True:
                curr_hidden_nn_layer = self.bn[idx](curr_hidden_nn_layer)
            hidden_nn_layers.append(curr_hidden_nn_layer)
            layer_idx += 1
            last_layer_size = layer_size
        nn_output=self.last_out(hidden_nn_layers[-1])

        return nn_output

class attention_fcn(tf.keras.layers.Layer):
    def __init__(self):
        super(attention_fcn, self).__init__()
        self.attention_mat = tf.keras.layers.Dense(hidden_size,use_bias=False)#[query_size]
        
        self._fcn_net=fcn_net(layer_sizes=[80,40])
    def call(self, query, key_value, return_alpha=False):
        att_inputs = self.attention_mat(key_value)
        if query.shape.ndims != att_inputs.shape.ndims:
            queries = tf.reshape(
                tf.tile(query, [1, tf.shape(att_inputs)[1]]), tf.shape(att_inputs)
            )
        else:
            queries = query
        last_hidden_nn_layer = tf.concat(
            [att_inputs, queries, att_inputs - queries, att_inputs * queries], -1
        )

        att_fnc_output = self._fcn_net(last_hidden_nn_layer)

        att_fnc_output = tf.squeeze(att_fnc_output, -1)

        att_weights = tf.nn.softmax(att_fnc_output
        )

        output = key_value * tf.expand_dims(att_weights, -1)
        if not return_alpha:
            return output
        else:
            return output, att_weights

In [10]:
class Bilinear(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Bilinear, self).__init__()
        self.linear_act = tf.keras.layers.Dense(units, activation=None, use_bias=True)
        self.linear_noact = tf.keras.layers.Dense(units, activation=None, use_bias=False)

    def call(self, a, b, gate_b=None):
        if gate_b is None:
            return tf.nn.sigmoid(self.linear_act(a) + self.linear_noact(b))
        else:
            return tf.nn.tanh(self.linear_act(a) + tf.math.multiply(gate_b, self.linear_noact(b)))

class MIGRU(tf.keras.layers.Layer):
    def __init__(self, units):
        ###-------------------------------###

    def call(self, inputs, state, att_score):
        ###-------------------------------###
        return final

In [11]:
class mgnm(tf.keras.models.Model):
    def __init__(self):
        super(mgnm, self).__init__()
        blocks=1
        embed_reg=0.
        layer_norm_eps=1e-6
        num_heads=1
        use_l2norm=False
        
        self.len_seq=len_seq
        self.item_embedding = tf.keras.layers.Embedding(items_num+1,hidden_size,input_length=self.len_seq,
                                                       embeddings_initializer='random_normal',
                                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.pos_embedding = tf.keras.layers.Embedding(len_seq,hidden_size,input_length=1,
                                                       embeddings_initializer='random_normal',
                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.dropout = tf.keras.layers.Dropout(1-keep_rate)
        self.hiera=hierarchical_interest(hidden_size,0)

        self.multi_interest_dict={}
        for i in range(0,2*(layers_num+1)):
            self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w1']=tf.keras.layers.Dense(hidden_size * 4, activation='tanh')
            self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w2']=tf.keras.layers.Dense(interest_num, activation=None)
        
        self.gru_encoder={}
        self.MIGRU_encoder={}
        self.attention_weight={}
        for i in range(0,2*(layers_num+1)):
            self.MIGRU_encoder['augru_encoder'+str(i)]=MIGRU(64)
            self.gru_encoder['gru_encoder'+str(i)] = tf.keras.layers.GRU(hidden_size,return_sequences=True)
            
        self.gcn=gcn_layer(hidden_size, layers_num)
        self.layer_weight=tf.Variable(initial_value=tf.ones((2*layers_num+1,1)),name='w_'+str(i-1),trainable=True)
        # norm
        self.use_l2norm = use_l2norm
    
    def calculate_score(self,att_outputs_dict,tar):
        ###-------------------------------###
        return score
        
        
    
    def call(self, inputs):
        seq_embed=self.item_embedding(inputs['click_seq'])
        ###-------------------------------###
        all_embeddings=self.gcn(adj,seq_embed)
        all_embeddings=self.hiera(all_embeddings)
        
        
        
        
        pos_encoding = tf.expand_dims(self.pos_embedding(tf.range(self.len_seq)), axis=0)  # (1, seq_len, embed_dim)
        att_outputs_dict={}
        
        for i,element in enumerate(all_embeddings):
            att_outputs_dict['embed_'+str(i)]=element+pos_encoding
            att_outputs_dict['embed_'+str(i)] = self.dropout(att_outputs_dict['embed_'+str(i)])
        
        pos_emb=self.item_embedding(tf.reshape(inputs['pos_item'], [-1, ]))#[b,h]
        pos_score=self.calculate_score(att_outputs_dict,pos_emb)
        
        if pred:
            neg_indx=tf.random.uniform([tf.shape(inputs['pos_item'])[0],test_neg_num],dtype=tf.int32,maxval=items_num,minval=1)
        else:
            neg_indx=tf.random.uniform([tf.shape(inputs['pos_item'])[0],neg_num],dtype=tf.int32,maxval=items_num,minval=1)
        
        neg_emb=self.item_embedding(neg_indx)
        neg_score=self.calculate_score(att_outputs_dict,neg_emb)
        
        logits = tf.nn.softmax(tf.concat([pos_score, neg_score], axis=-1))
        
        loss = tf.reduce_mean(- tf.math.log(logits)[:,0])+1e-6*self.adj_l1
        self.add_loss(loss)
        
        return logits
    
    def summary(self):
        inputs = {
            'click_seq': tf.keras.layers.Input(shape=(self.len_seq,), dtype=tf.int32),
            'pos_item': tf.keras.layers.Input(shape=(), dtype=tf.int32)
        }
        tf.keras.models.Model(inputs=inputs, outputs=self.call(inputs)).summary()

In [12]:
def hr(rank, k):
    """Hit Rate.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: hit rate.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1
    return res / len(rank)


def mrr(rank, k):
    """Mean Reciprocal Rank.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: mrr.
    """
    mrr = 0.0
    for r in rank:
        if r < k:
            mrr += 1 / (r + 1)
    return mrr / len(rank)


def ndcg(rank, k):
    """Normalized Discounted Cumulative Gain.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: ndcg.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1 / np.log2(r + 2)
    return res / len(rank)


def eval_rank(pred_y, metric_names, k=10):
    """Evaluate
        Args:
            :param pred_y: A ndarray.
            :param metric_names: A list like ['hr'].
            :param k: A scalar(int).
        :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    rank = pred_y.argsort().argsort()[:, 0]
    res_dict = {}
    for name in metric_names:
        if name == 'hr':
            res = hr(rank, k)
        elif name == 'ndcg':
            res = ndcg(rank, k)
        elif name == 'mrr':
            res = mrr(rank, k)
        elif name == 'precision':
            res = hr(rank, 1)
        else:
            break
        res_dict[name] = res
    return res_dict

def eval_pos_neg(model, test_data, metric_names, k=10, batch_size=None):
    """Evaluate the performance of Top-k recommendation algorithm.
    Note: Test data must contain some negative samples(>= k) and one positive samples.
    Args:
        :param model: A model built-by tensorflow.
        :param test_data: A dict.
        :param metric_names: A list like ['hr'].
        :param k: A scalar(int).
        :param batch_size: A scalar(int).
    :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    
    if len(test_data['users'])<30000:
        pred_y = - model.predict(test_data, batch_size)
        print(np.shape(pred_y))
        return eval_rank(pred_y, metric_names, k)
    else:
        final_dict={}
        for i in metric_names:
            final_dict.update({i:[]})
        for i in range(0,int(len(test_data['users'])/30000)+1):
            part_test_data_users=test_data['users'][i*30000:(i+1)*30000]
            part_test_data_poss=test_data['pos_item'][i*30000:(i+1)*30000]
            part_test_data_cli=test_data['click_seq'][i*30000:(i+1)*30000]
            part_test_data = {'users': np.array(part_test_data_users),'click_seq': np.array(part_test_data_cli),'pos_item': np.array(part_test_data_poss)}
            pred_y = - model.predict(part_test_data, batch_size)
            print(np.shape(pred_y))
            part_dict=eval_rank(pred_y, metric_names, k)
            for i in metric_names:
                final_dict [i].append(part_dict[i])
        for i in metric_names:
            final_dict [i]=np.mean(final_dict [i])
        return final_dict

In [13]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [14]:
from time import time
from tensorflow.keras.backend import clear_session
number=1
max_layers=10
k=[5,10,20]
ave_dict_list=[{'hr':[],'mrr':[],'ndcg':[],'precision':[]} for _ in range(len(k))]
t1 = time()
strategy = tf.distribute.MirroredStrategy()
batch_size=batch_size*strategy.num_replicas_in_sync
for i in range(number):
    clear_session()
    with strategy.scope():
        pred=0
        indx=1
        model = mgnm()
        #model.summary()
        optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
        model.compile(optimizer=optimizer)
        model.fit(
                x=train_data,
                epochs=epoch_num,
                verbose=1,
                batch_size=batch_size,
            callbacks=callback,
            validation_data=val_data)
        
    pred=1
    indx=0
    for idx,i in enumerate(k):
        eval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg','precision'], i, batch_size)
        for m in ['hr', 'mrr', 'ndcg','precision']:
            ave_dict_list[idx][m].append(eval_dict[m])
        print('K=%d, Precision = %.4f, HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (i, eval_dict['precision'], eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))
t2 = time()
for idx,i in enumerate(k):
    print('K=%d, Fit [%.1f s]: Precision = %.4f, HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (i, t2 - t1, np.mean(ave_dict_list[idx]['precision']),np.mean(ave_dict_list[idx]['hr']), np.mean(ave_dict_list[idx]['mrr']), np.mean(ave_dict_list[idx]['ndcg'])))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/100
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica